In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import tqdm as notebook_tqdm
import pandas as pd
import datetime
import time
import PIL
import yaml

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

/home/christian.foley/.conda/envs/pt_tf_cross/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [5]:
# Add module path to sys
module_path = os.path.abspath(os.path.join('..'))
print("System path: "+module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
import micro_dl.torch_unet.networks.Unet2D as Unet2D
import micro_dl.torch_unet.networks.Unet25D as Unet25D
import micro_dl.torch_unet.networks.layers.ConvBlock2D as ConvBlock2D
import micro_dl.torch_unet.networks.layers.ConvBlock3D as ConvBlock3D
import micro_dl.torch_unet.utils.dataset as ds
import micro_dl.torch_unet.utils.io as io
import micro_dl.torch_unet.utils.training as training 
import micro_dl.torch_unet.utils.inference as inference

import micro_dl.cli.preprocess_script as preprocess

import micro_dl.utils.aux_utils as aux_utils
import micro_dl.utils.train_utils as train_utils
import micro_dl.utils.preprocess_utils as preprocess_utils


System path: /home/christian.foley/virtual_staining/microDL


Using TensorFlow backend.


In [6]:
train_config_file_path = '../micro_dl/config_train_25D_torch.yml'

In [7]:
torch_data_container = ds.TorchDataset(train_config_file_path, 
                                       transforms = [ds.ToTensor()],
                                       target_transforms = [ds.ToTensor(), 
                                                            ds.GenerateMasks(masking_type = 'rosin', clipping = True, clip_amount = (10,0))]) #good practice to put GenerateMasks last in the list of sequential transforms
print(len(torch_data_container))

train_dataset = torch_data_container['train']
test_dataset = torch_data_container['test']
val_dataset = torch_data_container['val']
print(len(train_dataset), len(test_dataset), len(val_dataset))

3
173 20 25


In [8]:
val_dataloader = torch.utils.data.DataLoader(dataset = val_dataset, batch_size = 1, shuffle=True)

In [9]:
#3.) TESTING CYCLE
testing_cycle = training.run_test
testing_cycle

<function micro_dl.torch_unet.utils.training.run_test(test_dataloader, model, criterion, mask=True, plot=True, plot_num=2, epoch=None, save_folder=None, writer=None, device=device(type='cuda'))>

In [14]:
#2.) MODEL/OPT DEFINITION

#hyperparameters
params = {
    'model': {
        'architecture': '2.5D',
        'in_channels': 1,
        'out_channels': 1,
        'residual': True,
        'task': 'reg', #regression
        'device': torch.device('cuda'),
        'conv_mode': 'valid'
    },
    'training': {
        'epochs': 100,
        'learning_rate': 0.0003,
        'optimizer': 'adam',
        'loss': nn.MSELoss,
        'optimizer': optim.Adam,
        'testing_stride': 1
    }
}

In [15]:
model = Unet25D.Unet25d(in_channels = params['model']['in_channels'], out_channels = params['model']['out_channels'], 
                        residual = params['model']['residual'], task = params['model']['task'], conv_mode = params['model']['conv_mode'])
silent = model.cuda()

In [16]:
#define loss criterion and optimizer
criterion = params['training']['loss']()
optimizer = params['training']['optimizer'](model.parameters(), lr = params['training']['learning_rate'])

In [68]:
testing_cycle(val_dataloader, model, criterion, plot = True, plot_num = 6, mask = True)

input 0 shape: torch.Size([16, 1, 5, 256, 256])____________________] (4%)
output 0 shape: torch.Size([16, 16, 5, 254, 254])
input 1 shape: torch.Size([16, 16, 5, 254, 254])
output 1 shape: torch.Size([16, 16, 5, 252, 252])
input 2 shape: torch.Size([16, 16, 5, 252, 252])
output 2 shape: torch.Size([16, 16, 5, 250, 250])
torch.Size([16, 16, 5, 125, 125])
input 0 shape: torch.Size([16, 16, 5, 125, 125])
output 0 shape: torch.Size([16, 32, 5, 123, 123])
input 1 shape: torch.Size([16, 32, 5, 123, 123])
output 1 shape: torch.Size([16, 32, 5, 121, 121])
input 2 shape: torch.Size([16, 32, 5, 121, 121])
output 2 shape: torch.Size([16, 32, 5, 119, 119])
torch.Size([16, 32, 5, 59, 59])
input 0 shape: torch.Size([16, 32, 5, 59, 59])
output 0 shape: torch.Size([16, 64, 5, 57, 57])
input 1 shape: torch.Size([16, 64, 5, 57, 57])
output 1 shape: torch.Size([16, 64, 5, 55, 55])
input 2 shape: torch.Size([16, 64, 5, 55, 55])
output 2 shape: torch.Size([16, 64, 5, 53, 53])
torch.Size([16, 64, 5, 26, 26]

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 28 but got size 53 for tensor number 1 in the list.

In [64]:
z = torch.rand(1,1, 5,256,256)

In [65]:
ks = (1,3,3)
con1 = nn.Conv3d(1, 1, kernel_size = ks, padding = (ks[0]//2, 0, 0))

In [66]:
z_out = con1(con1(con1(z)))
z_out.shape

torch.Size([1, 1, 5, 250, 250])